## Fitness Measure

What we need is a way of determining if a piece of text which we decrypted is very similar to English. This is achieved by counting 'quadgrams' (also known as 'tetragraphs'), or groups of 4 letters.

e.g. the quadgrams in the text `ATTACK` are: `ATTA`, `TTAC`, and `TACK`. (single letter frequencies, bigrams and trigrams can also be used for this purpose.)
For the text `ATTACK`, the quadgrams are `ATTA`, `TTAC`, and `TACK`. The total probability is

`P(ATTACK) = p(ATTA)*p(TTAC)*p(TACK)`

`p(ATTA) = count(ATTA)/N`

`count(ATTA)` is number of times the particular quandgram occured

`N` is the total number of quadgrams in the trainning sample

### *fitness* 
Log probability of a piece of text, a higher number means it is more likely to be English, while a lower number means it is less likely to be English.

`log(p(ATTACK)) = log(p(ATTA)) + log(p(TTAC)) + log(p(TACK))`




In [19]:
from math import log10

class ngram_score(object):
    def __init__(self,ngramfile,sep=' '):
        ''' load a file containing ngrams and counts, calculate log probabilities '''
        self.ngrams = {}
        for line in open(ngramfile):
            key,count = line.split(sep) 
            self.ngrams[key] = int(count)
        self.L = len(key)
        self.N = sum(self.ngrams.values())
        #calculate log probabilities
        for key in self.ngrams.keys():
            self.ngrams[key] = log10(float(self.ngrams[key])/self.N)
        self.floor = log10(0.01/self.N)

    def score(self,text):
        ''' compute the score(fitness) of text '''
        score = 0
        ngrams = self.ngrams.__getitem__
        for i in range(len(text)-self.L+1):
            if text[i:i+self.L] in self.ngrams: score += ngrams(text[i:i+self.L])
            else: score += self.floor          
        return score
       


In [23]:
import re

fitness = ngram_score('english_quadgrams.txt') # load our quadgram statistics
from pycipher import Caesar
      
def break_caesar(ctext):
    # make sure ciphertext has all spacing/punc removed and is uppercase
    ctext = re.sub('[^A-Z]','',ctext.upper())
    # try all possible keys, return the one with the highest fitness
    scores = []
    for i in range(26):
        scores.append((fitness.score(Caesar(i).decipher(ctext)),i))
    return max(scores)
    
# example ciphertext
ctext = 'YMJHFJXFWHNUMJWNXTSJTKYMJJFWQNJXYPSTBSFSIXNRUQJXYHNUMJWX'
max_key = break_caesar(ctext)

print('best candidate with key (a,b) = '+str(max_key[0])+':')
print(Caesar(max_key[1]).decipher(ctext))





best candidate with key (a,b) = -227.95358542943333:
THECAESARCIPHERISONEOFTHEEARLIESTKNOWNANDSIMPLESTCIPHERS


# Mono-substitution cipher

In [2]:
from pycipher import SimpleSubstitution
ss = SimpleSubstitution('phqgiumeaylnofdxjkrcvstzwb')
ss.encipher('defend the east wall of the castle')


'GIUIFGCEIIPRCTPNNDUCEIQPRCNI'

In [3]:
ss.decipher('GIUIFGCEIIPRCTPNNDUCEIQPRCNI')

'DEFENDTHEEASTWALLOFTHECASTLE'

# Break Mono-substitution cipher

# `Hill-climbing` Algorithm
The hill-climbing algorithm looks like this:

1. Generate a random key, called the 'parent', decipher the ciphertext using this key. Rate the fitness of the deciphered text, store the result.
2. Change the key slightly (swap two characters in the key at random), measure the fitness of the deciphered text using the new key.
3. If the fitness is higher with the modified key, discard our old parent and store the modified key as the new parent.
4. Go back to 2, unless no improvement in fitness occurred in the last 1000 iterations.

As this cycle proceeds, the deciphered text gets fitter and fitter, the key becomes better until either the solution appears, or, the solution is not found. In this case the run has failed and must be repeated with a different starting key. This means the hill-climbing algorithm is stuck in a 'local maximum', where there are no simple changes that can be made to the key to improve fitness, and yet it is not at the true solution. If this happens you can run the algorithm again with a different parent in the hope it may reach the true solution this time. In the implementation below, we may restart the algorithm 100's of times in the search for the best key.

In [ ]:
import random
import re

ctext="""SOWFBRKAWFCZFSBSCSBQITBKOWLBFXTBKOWLSOXSOXFZWWIBICFWUQLRXINOCIJLWJFQUNWXLFBSZXFBT
XAANTQIFBFSFQUFCZFSBSCSBIMWHWLNKAXBISWGSTOXLXTSWLUQLXJBUUWLWISTBKOWLSWGSTOXLXTSWL
BSJBUUWLFULQRTXWFXLTBKOWLBISOXSSOWTBKOWLXAKOXZWSBFIQSFBRKANSOWXAKOXZWSFOBUSWJBSBF
TQRKAWSWANECRZAWJ
"""
ctext = re.sub('[^A-Z]','',ctext.upper())

maxkey = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
maxscore = -99e9
parentscore,parentkey = maxscore,maxkey[:]
print("Substitution Cipher solver, you may have to wait several iterations")
print ("for the correct result. Press ctrl+c to exit program.")
# keep going until we are killed by the user
i = 0
while 1:
    i = i+1
    random.shuffle(parentkey)
    deciphered = SimpleSub(parentkey).decipher(ctext)
    parentscore = fitness.score(deciphered)
    count = 0
    while count < 100:
        a = random.randint(0,25)
        b = random.randint(0,25)
        child = parentkey[:]
        # swap two characters in the child
        child[a],child[b] = child[b],child[a]
        deciphered = SimpleSub(child).decipher(ctext)
        score = fitness.score(deciphered)
        # if the child was better, replace the parent with it
        if score > parentscore:
            parentscore = score
            parentkey = child[:]
            count = 0
        count = count+1
    # keep track of best score seen so far
    if parentscore>maxscore:
        maxscore,maxkey = parentscore,parentkey[:]
        print ('\nbest score so far:',maxscore,'on iteration',i)
        ss = SimpleSub(maxkey)
        print ('    best key: '+''.join(maxkey))
        print ('    plaintext: '+ss.decipher(ctext))



# GA 
The following is an outline of proposed algorithm:
1. Input the cipher text to the algorithm and relative character frequencies.
2. Initialize the algorithm parameters: maximum number of generations (M).
3. Generate the population p(0) keys randomly (for example 20 keys) each one with length of 26 letters.
4. For 1 to (M) do:
    a. Decrypt the cipher text by the 20 generated keys.

    b. Calculate the suitability of each key from every decrypted text using the formula of fitness.

    c. Sort the keys based on the increased fitness values.

    d. Keep 20% (2 pairs) of best fittest of p(0) for next generation.

    e. Use stochastically selection to choose 8 pairs from the 20 keys (parents).

    f. For 1 to 10 pairs do: 

        i. Apply crossover to get children (10 pairs).

        ii. Apply mutation of 0.02% to the new children.

        iii. Apply replacement to get a new population (20 keys).

g. Go to 4.

5. Output is the best solution. 

Cuộc tấn công được thực hiện bằng cách tạo ra một nhóm khóa ứng cử viên ban đầu p (0) về số lượng thẻ chẵn, bao gồm các hoán vị của tập {a, b, c ,. . . z}. Thế hệ đầu tiên được tạo ngẫu nhiên bằng cách sử dụng một trình tạo ngẫu nhiên thống nhất đơn giản.
Sau đó, văn bản mật mã được giải mã bằng cách sử dụng mỗi hoán vị làm khóa, cho phép chúng ta gán một số đo thể lực bằng cách sử dụng phương trình (1) cho mỗi khóa ứng viên. Các cặp khóa ứng viên sau đó được chọn ngẫu nhiên để tạo ra con cái sau khi áp dụng phương pháp trao đổi chéo cho mỗi cặp.
 Phương pháp lựa chọn ngẫu nhiên được áp dụng bằng cách chọn một số cặp từ ứng cử viên mà họ có thể lực tốt nhất.
Sau đó, Stochastic Universal Sampling hoạt động bằng cách tạo một vòng quay duy nhất của bánh xe roulette. Điều này cung cấp một vị trí bắt đầu và cá nhân được chọn đầu tiên. Quá trình lựa chọn sau đó tiến hành bằng cách tiến tất cả các cách xung quanh bánh xe theo các bước có kích thước bằng nhau, trong đó kích thước bước được xác định bởi số lượng cá nhân được chọn. Vì vậy, nếu chúng tôi đang chọn n cá nhân, chúng tôi sẽ tăng thêm 1 / n x 360 độ cho mỗi lựa chọn. Lưu ý rằng điều này không có nghĩa là mọi ứng cử viên trên bánh xe sẽ được chọn. Một số cá nhân yếu sẽ có
các lát rất mỏng của bánh xe và chúng có thể được chuyển hoàn toàn tùy thuộc vào vị trí bắt đầu ngẫu nhiên, như trong Hình.5 [12].

Crossover là quá trình lấy hai giải pháp cha mẹ và sản xuất từ ​​chúng một đứa trẻ. Sau quá trình lựa chọn, dân số được làm giàu với các cá nhân tốt hơn. Crossover là một toán tử tái hợp tiến hành theo ba bước:
Tôi. Toán tử sinh sản chọn ngẫu nhiên một cặp hai khóa riêng lẻ để giao phối.
ii. Một trang web chéo được chọn ngẫu nhiên dọc theo chiều dài khóa.
iii. Cuối cùng, các giá trị vị trí được hoán đổi giữa hai khóa theo trang web chéo.
Thuật toán di truyền truyền thống sử dụng trao đổi điểm đơn (được sử dụng trong bài báo này), khi hai nhiễm sắc thể giao phối được cắt một lần tại các điểm tương ứng và các phần sau khi các vết cắt trao đổi. Ở đây, một trang web chéo hoặc điểm giao nhau được chọn ngẫu nhiên dọc theo chiều dài của khóa giao phối và các chữ cái bên cạnh các trang web chéo được trao đổi. Nếu bất kỳ nhiễm sắc thể trao đổi nào đã xuất hiện ở trẻ, thì các vị trí nhiễm sắc thể này sẽ bị bỏ trống, sau đó các chữ cái không xuất hiện ở trẻ được chèn vào đó. Kết quả của thủ tục này được thể hiện trong Hình.6 [12, 18,19].